# Most Reliable Path

Most Reliable Path (MRP) is a variant of the Shortest Path Problem, where, each edge is
randomly associated with probabilities (0.512 or 0.8) which denotes the “reliability” of the edge,
and the task is to find the most reliable path between the source and the destination node. We
do not remove edges this time.

We use the same 5-layer MLP in the SP experiment as the baseline. We also use the simplepath and the reachability constraints to train the neural network by NeurASP. Besides, we
use weak constraints to represent the probability of each edge in the grid.

## Data Format

In dataGen.py, a class named "GridProbData" is defined in the following way.

GridProbData class has 6 attributes: train_data, test_data, valid_data, train_labels, test_labels, valid_labels.

train_data is an numpy array of size (1800, 40). It consists of 1800 data as follows 

        [  
          data,  
          ...,  
          data  
        ]  

where data is a vector (numpy array) of length 40. For example, the data shown below  

        [  
          0.512 0.8 0.512 0.8 0.512  
          0.512 0.8 0.512 0.512 0.8  
          0.8 0.512 0.512 0.8 0.512  
          0.512 0.512 0.512 0.512 0.8  
          0.512 0.8 0.512 0.8  
          10000 00000 01000 0  
        ]  

defines the 24 probabilities of the 24 edges and specified that the nodes 0 and 11 are the starting and ending nodes.  
train_labels is an numpy array of size (1800, 24). It consists of 1800 label as follows.  

        [  
          label,  
          ...,  
          label  
        ]  

where label is a vector (numpy array) of length 24. For example, the label shown below  

        [11100 00000 00000 00000 0110]  

means that the edges 0, 1, 2, 21, 22 form a most reliable path.  
test_data is a numpy array of size (600, 40).  
valid_data is a numpy array of size (600, 40).  
test_labels is a numpy array of size (600, 24).  
valid_labels is a numpy array of size (600, 24).  



## Imports

In [1]:
import sys
sys.path.append("../../")
import random
import time

import numpy as np
import torch
from torch.autograd import Variable

from dataGen import GridProbData
from neurasp import NeurASP
from network import FC

there are 3000 data in total, 60% training data, 20% validation data, 20% testing data!
done!


In [2]:

import os,sys,inspect
currentdir = os.path.dirname(os.path.abspath(inspect.getfile(inspect.currentframe())))
parentdir = os.path.dirname(currentdir)
sys.path.insert(0,parentdir) 




## NeurASP Programs for Training and Testing

In [3]:
dprogram = '''
nn(mrp(24, g), [true, false]).
'''

aspProgram = '''
mrp(X) :- mrp(X,g,true).
mrp(0,1) :- mrp(0).
mrp(1,2) :- mrp(1).
mrp(2,3) :- mrp(2).
mrp(4,5) :- mrp(3).
mrp(5,6) :- mrp(4).
mrp(6,7) :- mrp(5).
mrp(8,9) :- mrp(6).
mrp(9,10) :- mrp(7).
mrp(10,11) :- mrp(8).
mrp(12,13) :- mrp(9).
mrp(13,14) :- mrp(10).
mrp(14,15) :- mrp(11).
mrp(0,4) :- mrp(12).
mrp(4,8) :- mrp(13).
mrp(8,12) :- mrp(14).
mrp(1,5) :- mrp(15).
mrp(5,9) :- mrp(16).
mrp(9,13) :- mrp(17).
mrp(2,6) :- mrp(18).
mrp(6,10) :- mrp(19).
mrp(10,14) :- mrp(20).
mrp(3,7) :- mrp(21).
mrp(7,11) :- mrp(22).
mrp(11,15) :- mrp(23).
mrp(X,Y) :- mrp(Y,X).
:- X=0..15, #count{Y: mrp(X,Y)} = 1.
:- X=0..15, #count{Y: mrp(X,Y)} >= 3.
reachable(X, Y) :- mrp(X, Y).
reachable(X, Y) :- reachable(X, Z), mrp(Z, Y).
:- mrp(X, _), mrp(Y, _), not reachable(X, Y).
'''

## Neural Network Instantiation
- Instantiate neural networks.
- Define nnMapping: a dictionary that maps neural network names (i.e., strings) to the neural network objects (i.e., torch.nn.Module object)
- Define optimizers: a dictionary that specifies the optimizer for each network (we use the Adam optimizer here).

In [4]:
m = FC(40, *[50, 50, 50, 50, 50], 24)
nnMapping = {'mrp': m}
optimizer = {'mrp': torch.optim.Adam(m.parameters(), lr=0.001)}

Neural Network (MLP) Structure: (40, 50, 50, 50, 50, 50, 24)


## Create NeurASP object

In [5]:
NeurASPobj = NeurASP(dprogram+aspProgram, nnMapping, optimizer)

## Create dataList and obsList for Training, testDataList and testObsList for Testing
### Create the dataset object

In [6]:
dataset = GridProbData("data/data.txt")

there are 3000 data in total, 60% training data, 20% validation data, 20% testing data!


### Construct dataList and obsList

In [7]:
dataList = []
obsList = []

for i, d in enumerate(dataset.train_data):
    d_tensor = Variable(torch.from_numpy(d).float(), requires_grad=False)
    dataList.append({'g': d_tensor})

with open('data/evidence_train.txt', 'r') as f:
    obsList = f.read().strip().strip('#evidence').split('#evidence')

### Construct testDataList and testObsList

In [8]:
dataListTest = []
obsListTest = []

for d in dataset.test_data:
    d_tensor = Variable(torch.from_numpy(d).float(), requires_grad=False)
    dataListTest.append({'g': d_tensor})

with open('data/evidence_test.txt', 'r') as f:
    obsListTest = f.read().strip().strip('#evidence').split('#evidence')

## Training and Testing

Note that our target is to find the path with the highest probability, which is represented by the optimal stable models of the logic program. To find the optimal stable models instead of stable models during training, we need to specify "opt=True" in the learning function.

In [9]:
for i in range(20):
    print('Continuously training for 10 epochs round {}...'.format(i+1))
    time1 = time.time()
    NeurASPobj.learn(dataList=dataList, obsList=obsList, epoch=10, opt=True, smPickle='data/stableModels.pickle')
    time2 = time.time()
    NeurASPobj.testConstraint(dataList=dataListTest, obsList=obsListTest, mvppList=[aspProgram])
    print("--- train time: %s seconds ---" % (time2 - time1))
    print("--- test time: %s seconds ---" % (time.time() - time2))
    print('--- total time from beginning: %s minutes ---' % int((time.time() - startTime)/60) )

print('Storing the trained model into {}'.format(saveModelPath))
torch.save(m.state_dict(), saveModelPath)

Continuously training for 10 epochs round 1...
The accuracy for constraint 1 is 0.0
--- train time: 155.37194752693176 seconds ---
--- test time: 9375.305091142654 seconds ---


NameError: name 'startTime' is not defined